In [ ]:
from openpyxl import load_workbook
wb = load_workbook('exercises/data/f1-results.xlsx')


In [ ]:
import os
os.system("open exercises/data/f1-results.xlsx")

**Zadanie 1:** Przygotuj arkusz o nazwie *Standings*, który będzie zawierał sumaryczną liczbę punktów zawodników po każdym wyścigu. 
    
Arkusz powinien zawierać tabelą jak poniżej:

|                  | Australia | China | ... |
|------------------|-----------|-------|-----|
| Lewis Hamilton   | 18        | 43    | ... |
| Sebastian Vettel | 25        | 43    | ... |
| ...              |           |       |     |

**Zadanie 2:** Na podstawie powyższej tabeli, w arkuszu *Standings* umieść wykres liniowy (`LineChart`), który zaprezentuje sumaryczną liczbę punktów po kolejnych wyścigach. Na osi X powinny być nazwy wyścigów, na osi Y liczba punktów, a serie reprezentują poszczególnych kierowców. 


In [ ]:
# Zadanie 1

In [ ]:
# Zadanie 2

In [2]:
# Przyklad z: https://realpython.com/openpyxl-excel-spreadsheets-python/

import random
from openpyxl import Workbook
from openpyxl.chart import LineChart, Reference

workbook = Workbook()
sheet = workbook.active

# Let's create some sample sales data
rows = [
    ["", "January", "February", "March", "April",
    "May", "June", "July", "August", "September",
     "October", "November", "December"],
    [1, ],
    [2, ],
    [3, ],
]

for row in rows:
    sheet.append(row)

for row in sheet.iter_rows(min_row=2,
                           max_row=4,
                           min_col=2,
                           max_col=13):
    for cell in row:
        cell.value = random.randrange(5, 100)

# Create a LineChart and add the main data
chart = LineChart()
data = Reference(worksheet=sheet,
                           min_row=2,
                           max_row=4,
                           min_col=1,
                           max_col=13)
chart.add_data(data, titles_from_data=True, from_rows=True)

# Add categories to the chart
cats = Reference(worksheet=sheet,
                 min_row=1,
                 max_row=1,
                 min_col=2,
                 max_col=13)
chart.set_categories(cats)

# Rename the X and Y Axis
chart.x_axis.title = "Months"
chart.y_axis.title = "Sales (per unit)"

# Apply a specific Style
chart.style = 24

# Save!
sheet.add_chart(chart, "C6")
workbook.save("./exercises/tmp/line_chart.xlsx")

In [4]:
import os
os.system("open ./exercises/tmp/line_chart.xlsx")

0